In [100]:
import math
import numpy as np
import sys
import json
import numpy as np
import functools
from typing import List, Callable
import numpy as np
from scipy.interpolate import BSpline
from scipy.optimize import minimize
from scipy.integrate import quad


In [101]:
#Read & parse Option file

with open('/home/cordeau/Cours/etude_article/basket_d5.txt', 'r') as file:
    content = file.readlines()
# Initialisation des variables
rho = degree = T = size = dates = n_samples = 0
spot = sigma = divid = coeffs = None
type = ""
strike = 0.0
opt = None
c = False
# Extraction des paramètres
for line in content:
    ine = line.strip()
    if "correlation" in line:
        rho = float(line.split()[-1])
    elif "degree for polynomial regression" in line:
        degree = int(line.split()[-1])
    elif "option type" in line:
        type = line.split()[-1].strip()
    elif "maturity" in line:
        T = float(line.split()[-1])
    elif "model size" in line:
        size = int(line.split()[-1])
    elif "dates" in line:
        dates = int(line.split()[-1])
    elif "spot" in line:
        spot =[float(line.split()[-1])]
    elif "volatility" in line:
        sigma = [float(line.split()[-1])]
    elif "interest rate" in line:
        r = float(line.split()[-1])
    elif "dividend rate" in line:
        divid = [float(line.split()[-1])]
    elif "strike" in line:
        strike = float(line.split()[-1])
    elif "MC iterations" in line:
        n_samples = int(line.split()[-1])
    elif "payoff coefficients" in line:
        coeffs = [float(line.split()[-1])]


#Create the payoff function
def payoffVect(St):
    sum = np.dot(coeffs, St)
    sum -= strike

    return np.maximum(0, sum)

payoffVect(80)


array([20.])

In [102]:
nbTimeSteps = 10
step = T/nbTimeSteps

def asset(CorrelationMatrix, path):
    Gaussian_k = np.zeros(size)
    CorrelationLine = np.zeros(size)
    div = 0.0
    sig = 0.0
    expon = 0.0
    value = 0.0

    path[0,:] = spot
    for k in range(1, nbTimeSteps):
        Gaussian_k = np.random.normal(0, 1, size)
        for d in range(size):
            div = divid[d]
            sig = sigma[d]
            CorrelationLine = CorrelationMatrix[d,:]
            expon = math.exp((r - div - sig*sig/2.0)*step + sig*math.sqrt(step)*np.dot(CorrelationLine, Gaussian_k))
            value = expon*path[k-1,d]
            path[k,d] = value

    del CorrelationLine, Gaussian_k

path = np.zeros((nbTimeSteps, 1))
correl = np.zeros((1,1))
correl[0][0] = 1
asset(correl, path)
print(path)

[[100.        ]
 [ 86.4763463 ]
 [ 78.4636626 ]
 [ 80.20759251]
 [ 78.81353482]
 [ 80.34043158]
 [ 75.40135103]
 [ 79.95847744]
 [ 92.63073004]
 [ 93.76488213]]


In [103]:
#Creation of differents paths.
allPath = np.empty(n_samples, dtype=object)
for i in range(n_samples):
    path = np.zeros((nbTimeSteps, 1))
    asset(correl, path)
    allPath[i] = path


In [104]:
#Creation of the orthogonal base for the inner product of the article
K = 3

def polynom_result(vector, x):
    res = 0
    for i in range(len(vector)):
        res += (x**i)*vector[i]
    return res

def norm(vector, points):
    res = 0
    for i in range(len(points)):
        for j in range(len(points[0])):
            res += polynom_result(vector, math.log(points[i][j]))
    return math.sqrt(abs(res))

def inner_product(u, v, matrice):
    #print("innerprod")
    # Use the polarization identity to compute the inner product
    u_plus_v_norm_sq = norm(u + v, matrice)
    u_minus_v_norm_sq = norm(u - v, matrice)
    result = (u_plus_v_norm_sq * u_plus_v_norm_sq - u_minus_v_norm_sq * u_minus_v_norm_sq) / 4.0
    
    return result


def gram_schmidt():

    A = np.identity(K+1)
    Q = np.zeros_like(A)
    for i in range(A.shape[0]):
        # Take the i-th vector
        v = A[i]
        # Subtract the projection of v onto the span of the previous orthonormal vectors
        for j in range(i):
            v -= inner_product(Q[j], A[i], allPath) * Q[j]
        # Normalize the resulting vector
        Q[i] = v / norm(v, allPath)
    return Q

base = gram_schmidt()
print(base)


[[ 0.05773503  0.          0.          0.        ]
 [-0.01423952  0.02847904  0.          0.        ]
 [-0.00333604 -0.00667207  0.01334415  0.        ]
 [-0.00077652 -0.00155304 -0.00310609  0.00621218]]


In [105]:
#Definition of differents functions to integrate

def Yt(path, t):
    index = int(t//step)
    return math.exp(-r * t) * payoffVect(path[index][:])

def Lambdat(polynom, t, path):
    res = 0
    index = int(t//step)
    rebased_polynom = np.dot(base, polynom)
    spot = path[index]
    p_x = polynom_result(rebased_polynom, math.log(spot))
    if(payoffVect(path[index][:]) > 0):
        res = math.exp(p_x)
    return res

def Ut(polynom, t, path):
    integration = 0
    for i in range(nbTimeSteps):
        integration += Lambdat(polynom, i*step, path) * step
    return math.exp(-integration)

pol = [1] * (K+1)
Ut(pol, 0.6, path)

0.40176702137001097

In [106]:
def phi(polynom, path):
    integration = 0
    for i in range(nbTimeSteps):
        integration += quad(lambda t: Yt(path, t) * Ut(polynom, t, path) * Lambdat(polynom, t, path), i*step, (i+1)*step)[0]
    return integration + Yt(path, T) * Ut(polynom, T, path)
phi(pol, path)

4.387400825782612

In [107]:
#Definition of the function tu optimizate
def psy(polynom, allpath):
    M = len(allPath)
    res = 0
    for path in allPath:
        res += phi(polynom, path)
    return res / M

psy(pol, allPath)

4.674365951610311

In [108]:
def optimization(coefficients, allPath):
    bounds = [(None, None)] * (K+1)
    result = minimize(lambda x: -psy(x, allPath), x0=coefficients, bounds=bounds, method='L-BFGS-B')
    return result

polynom_to_optimize = np.random.rand(K+1)  # point de départ initial

optimization(polynom_to_optimize, allPath)

  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: -6.95271727965168
        x: [ 4.732e+00  4.658e+00  7.879e-01 -1.638e+01]
      nit: 17
      jac: [-1.776e-06 -2.309e-06 -7.994e-07  8.171e-06]
     nfev: 90
     njev: 18
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>

In [109]:
optimized = np.array([4.732e+00 ,4.658e+00 ,7.879e-01 ,-1.638e+01])
asset(correl, path)
print(psy(optimized, path))

5.844783580830016
